In [1]:
import pandas as pd


In [2]:
df_features = pd.read_csv('../data/features.csv')
df_sales = pd.read_csv('../data/sales.csv')
df_stores = pd.read_csv('../data/stores.csv')

In [3]:
df_features['Date'] = pd.to_datetime(df_features['Date'], format='%d/%m/%Y')
df_sales['Date'] = pd.to_datetime(df_sales['Date'], format='%d/%m/%Y')


In [4]:
df_features = df_features.rename(columns={'IsHoliday': 'IsHolidayFeatures'})
df_combined = pd.merge(df_sales, df_stores, on='Store', how='left')
df_final = pd.merge(df_combined, df_features, on=['Store', 'Date'], how='left')
df_final['week'] = df_final['Date'].dt.isocalendar().week
df_final['month'] = df_final['Date'].dt.month
df_final['day'] = df_final['Date'].dt.day
df_final['original_date'] = df_final['Date']
df_final.set_index('Date', inplace=True)
df_final = df_final.sort_index()


In [5]:
df_final.drop(columns=['IsHolidayFeatures'], axis=1, inplace=True)

In [6]:
shiftable_columns = ['Weekly_Sales', 'Temperature', 'MarkDown1','MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI','Unemployment']

In [7]:
for col in shiftable_columns:
    df_final[f'lag_{col}'] = df_final.groupby(['Store', 'Dept'])[f'{col}'].shift(52)
df_shifted = df_final.drop(shiftable_columns, axis = 1)
df_shifted =df_shifted.dropna(subset=['lag_Weekly_Sales'])
df_shifted.rename(columns={col: col[4:] for col in df_shifted.columns if col.startswith('lag_')}, inplace=True)


In [8]:
df_shifted.to_csv('improved_data.csv', index=False)


In [10]:
df_shifted

,Store,Dept,IsHoliday,Type,Size,Fuel_Price,week,month,day,original_date,Weekly_Sales,Temperature,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
Date,,,,,,,,,,,,,,,,,,,
2011-02-04,1,81,False,A,151315,2.989,5,2,4,2011-02-04,30052.75,42.31,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
2011-02-04,2,56,False,A,202307,2.989,5,2,4,2011-02-04,2024.13,40.19,NaN,NaN,NaN,NaN,NaN,210.752605,8.324
2011-02-04,20,41,False,A,203742,3.231,5,2,4,2011-02-04,1193.96,25.92,NaN,NaN,NaN,NaN,NaN,204.247194,8.187
2011-02-04,8,23,False,A,155078,2.989,5,2,4,2011-02-04,15034.20,34.14,NaN,NaN,NaN,NaN,NaN,214.471451,6.299
2011-02-04,29,82,False,B,93638,3.240,5,2,4,2011-02-04,7432.97,24.36,NaN,NaN,NaN,NaN,NaN,131.527903,10.064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-10-26,40,87,False,A,155083,3.917,43,10,26,2012-10-26,20911.56,42.09,NaN,NaN,NaN,NaN,NaN,136.488452,4.420
2012-10-26,19,30,False,A,203819,4.071,43,10,26,2012-10-26,3673.78,46.49,NaN,NaN,NaN,NaN,NaN,136.488452,7.866
2012-10-26,19,31,False,A,203819,4.071,43,10,26,2012-10-26,3668.03,46.49,NaN,NaN,NaN,NaN,NaN,136.488452,7.866
